In [1]:
import pandas as pd
import torch

from torch.utils.data import Dataset

In [4]:
filtered_dataset = pd.read_csv('../data/interim/filtered.csv')
filtered_dataset.rename(columns={filtered_dataset.columns[0]: "id"}, inplace=True)

sorted = filtered_dataset.sort_values(by=['ref_tox'], ascending=False)
sorted.head()

,id,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
551255,551255,His father would have used a booming voice to ...,his father would have answered with his thunde...,0.729428,0.091954,0.999724,0.004599
101676,101676,You have to send those idiots back in.,you have to get those guys back there.,0.622852,0.000000,0.999723,0.000115
258368,258368,Salina could be with that stupid cop.,Salina could be with the cop.,0.774944,0.210526,0.999723,0.000500
318050,318050,And don't let those idiots in radiology hold y...,don't let them fool you in radiology.,0.711188,0.283019,0.999723,0.000874
70934,70934,My idiot friend here brought marijuana... - on...,my friend here took a marijuana...,0.715508,0.396552,0.999722,0.000161


In [5]:
class ToxicDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        ref = self.dataframe.iloc[idx, 1]
        trn = self.dataframe.iloc[idx, 2]

        return ref, trn

In [7]:
import datasets


torch_dataset = ToxicDataset(sorted)

def gen_train():
    with open('train_text.txt', 'w') as out_file:
        for idx in range(len(torch_dataset)):
            out_file.write(f"User: {torch_dataset[idx][0]}\nAssistant: {torch_dataset[idx][1]}\n\n")

/root/anaconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
gen_train()

In [26]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments


def fine_tune_gpt2(model_name, train_file, output_dir):
    # Load GPT-2 model and tokenizer
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    # Load training dataset
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_file,
        block_size=512)
    # Create data collator for language modeling
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False)
    # Set training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=2,
        per_device_train_batch_size=4,
        save_steps=1000,
        save_total_limit=2,
    )
    # Train the model
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )
    trainer.train()
    # Save the fine-tuned model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

In [27]:
fine_tune_gpt2("./output", "train_text.txt", "output")

/root/anaconda3/envs/torch/lib/python3.11/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,2.263200
1000,2.263900
1500,2.255500
2000,2.254600
2500,2.267000
3000,2.268500
3500,2.279600
4000,2.276600
4500,2.281800
5000,2.281500


In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained('./gpt2_output')
tokenizer = GPT2Tokenizer.from_pretrained('./gpt2_output')

/Users/maxim/mambaforge/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
def evaluate(s: str) -> str:
    # Convert string to array of embedding indices
    encoded_input = tokenizer(s, return_tensors='pt')
    # Pass embedding indices to the model
    output = model.generate(encoded_input.input_ids)
    # Decode resulting embedding indices back to string
    output_str = tokenizer.batch_decode(output, skip_special_tokens=True)
    
    output_str = output_str[0]

    # Truncate string to the next "User:" occurrence, if any.
    first_idx: int = output_str.find('User:')
    idx = output_str.rfind('User')
    if idx != -1 and idx != first_idx:
        output_str = output_str[:idx]
    return output_str

In [15]:
result = evaluate("User: shut up you moron. Fuck you.\nAssistant:")
print(result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: shut up you moron. Fuck you.
Assistant: you're kidding me.
